# Predicting antenna behaviour for curved surface obstacles using ML
 

In [1]:
#importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors              
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Loading Dataset

In [2]:
df = pd.read_excel('C:\\Users\\mypc\\Desktop\\BTP_B22DB04\\BTP_Dataset.xlsx',
                header=0)
df.head()



,Frequency(GHz) (f),Dipole_length (L),Impedance (z),Wire Radius(mm) (rho),Radius of curvature (m) (a),Curved surface height(m) (h),Angle(theta),Distance(m) (d),Relative_Permittivity,Max_gain,Direction_max_gain,Reflection_Pattern
0,1,0.0749,50,1,0.2997,0.1498,15,0.2997,infinite,2.763,310.0,Multi-Directional
1,1,0.0749,50,1,0.2997,0.1498,15,0.5995,infinite,2.119,332.0,Multi-Directional
2,1,0.0749,50,1,0.2997,0.1498,15,0.8993,infinite,1.943,338.1,Multi-Directional
3,1,0.0749,50,1,0.2997,0.1498,15,1.1991,infinite,1.865,342.0,Multi-Directional
4,1,0.0749,50,1,0.2997,0.1498,15,1.4989,infinite,1.819,344.0,Multi-Directional


In [3]:
df.dtypes

Frequency(GHz) (f)                int64
Dipole_length (L)               float64
Impedance (z)                     int64
Wire Radius(mm) (rho)             int64
Radius of curvature (m) (a)     float64
Curved surface height(m) (h)    float64
Angle(theta)                      int64
Distance(m) (d)                 float64
Relative_Permittivity            object
Max_gain                        float64
Direction_max_gain              float64
Reflection_Pattern               object
dtype: object

## Data Pre-Processing

### Getting Features and Target variables 

X holds all the features 

In [4]:
X=df.drop(['Max_gain', 'Direction_max_gain','Reflection_Pattern'],axis=1).copy()
X.head()

,Frequency(GHz) (f),Dipole_length (L),Impedance (z),Wire Radius(mm) (rho),Radius of curvature (m) (a),Curved surface height(m) (h),Angle(theta),Distance(m) (d),Relative_Permittivity
0,1,0.0749,50,1,0.2997,0.1498,15,0.2997,infinite
1,1,0.0749,50,1,0.2997,0.1498,15,0.5995,infinite
2,1,0.0749,50,1,0.2997,0.1498,15,0.8993,infinite
3,1,0.0749,50,1,0.2997,0.1498,15,1.1991,infinite
4,1,0.0749,50,1,0.2997,0.1498,15,1.4989,infinite


<br> y_1 is the target variable 'MAX_GAIN'

In [5]:
y_1=df['Max_gain'].copy()
y_1.head()

0    2.763
1    2.119
2    1.943
3    1.865
4    1.819
Name: Max_gain, dtype: float64

<br>y_2 is the target variable 'Direction of the Maximum Gain'

In [6]:
y_2=df['Direction_max_gain'].copy()
y_2.head()

0    310.0
1    332.0
2    338.1
3    342.0
4    344.0
Name: Direction_max_gain, dtype: float64

<br>y_3 is the target variable 'Reflection_Pattern'

In [7]:
y_3=df['Reflection_Pattern'].copy()
y_3.head()

0    Multi-Directional
1    Multi-Directional
2    Multi-Directional
3    Multi-Directional
4    Multi-Directional
Name: Reflection_Pattern, dtype: object

### Processing of the feature 'Relative Permittivity'

Since we input numbers to our Machine Learning models and to find a way to input the value of Infinity, we take the reciprocal of the feature 'Permittivity' for all the Data samples 

In [8]:
X['Relative_Permittivity'].head()


0    infinite
1    infinite
2    infinite
3    infinite
4    infinite
Name: Relative_Permittivity, dtype: object

In [9]:
def reciprocal(x):
    if x == 'infinite':
        return 0
    else:
        return 1/x

X['Relative_Permittivity'] = X['Relative_Permittivity'].apply(reciprocal)


In [10]:
X.head()

,Frequency(GHz) (f),Dipole_length (L),Impedance (z),Wire Radius(mm) (rho),Radius of curvature (m) (a),Curved surface height(m) (h),Angle(theta),Distance(m) (d),Relative_Permittivity
0,1,0.0749,50,1,0.2997,0.1498,15,0.2997,0
1,1,0.0749,50,1,0.2997,0.1498,15,0.5995,0
2,1,0.0749,50,1,0.2997,0.1498,15,0.8993,0
3,1,0.0749,50,1,0.2997,0.1498,15,1.1991,0
4,1,0.0749,50,1,0.2997,0.1498,15,1.4989,0


### Data Scaling

Since we are dealing with data of different ranges, we need to scale them so that they contribute in similar proportions.

In [11]:
X_scaled = scale(X)
X_encoded_df=pd.DataFrame(X_scaled)
X_encoded_df.head()

,0,1,2,3,4,5,6,7,8
0,0.0,1.387779e-17,0.0,0.0,-1.414214,2.775558e-17,-1.31122,-1.566699,0.0
1,0.0,1.387779e-17,0.0,0.0,-1.414214,2.775558e-17,-1.31122,-1.218544,0.0
2,0.0,1.387779e-17,0.0,0.0,-1.414214,2.775558e-17,-1.31122,-0.870388,0.0
3,0.0,1.387779e-17,0.0,0.0,-1.414214,2.775558e-17,-1.31122,-0.522233,0.0
4,0.0,1.387779e-17,0.0,0.0,-1.414214,2.775558e-17,-1.31122,-0.174078,0.0


In [12]:
mean_y_1=y_1.mean()
var_y_1=y_1.var()
print(var_y_1)
print(mean_y_1)

2.1288690982620078
2.7533348


In [13]:
mean_y_2=y_2.mean()
var_y_2=y_2.var()
print(var_y_2)
print(mean_y_2)

2333.4720419277105
320.79319999999996


In [14]:
y_1_scaled = scale(y_1)
#X_encoded_scaled
y_1_df=pd.DataFrame(y_1_scaled)
y_1_df.head()


,0
0,0.006638
1,-0.435627
2,-0.556494
3,-0.610060
4,-0.641650


In [15]:
y_2_scaled = scale(y_2)
#X_encoded_scaled
y_2_df=pd.DataFrame(y_2_scaled)
y_2_df.head()

,0
0,-0.223882
1,0.232461
2,0.358993
3,0.439890
4,0.481376


## Splitting Data into test and train splits (70% train and 30% test) for each of the target variables

In [16]:

X_train_1, X_test_1, y_train_1 , y_test_1 =train_test_split(X_scaled, y_1_scaled )



In [17]:
X_train_2, X_test_2, y_train_2 , y_test_2 =train_test_split(X_scaled, y_2_scaled )

In [18]:
X_train_3, X_test_3, y_train_3 , y_test_3 =train_test_split(X_scaled, y_3 )

# Building ML models and evaluating them 

## Regression Models to Predict the Max_Gain and Direction of Max_Gain 

### Support Vector Regression model with Linear kernel

<br>ML model : SVR with Linear kernel <br>
Prediction type: Regression <br>
Target Parameter: Maximum Gain <br>
Evaluation metrics: Mean Squared Error(MSE) , Mean absolute error(MAE)

In [19]:
reg_svm = SVR(kernel='linear')
reg_svm.fit(X_train_1, y_train_1)

#metrics

y_pred_1_l = reg_svm.predict(X_test_1)
mse_1_l=mean_squared_error(y_test_1, y_pred_1_l ,squared=True)
print('MSE for predicting the MAXIMUM GAIN using linear SVR is ' ,end='')
print('%.5f'%(mse_1_l))


mae_1_l= mean_absolute_error(y_test_1, y_pred_1_l)
print('MAE for predicting the MAXIMUM GAIN using linear SVR is ',end='')
print('%.5f'%(mae_1_l))

MSE for predicting the MAXIMUM GAIN using linear SVR is 1.04304
MAE for predicting the MAXIMUM GAIN using linear SVR is 0.51752


<br>ML model : SVR with Linear kernel <br>
Prediction type: Regression <br>
Target Parameter: Direction of Maximum Gain <br>
Evaluation metrics: Mean Squared Error(MSE) , Mean absolute error(MAE)

In [20]:
reg_svm = SVR(kernel='linear')
reg_svm.fit(X_train_2, y_train_2)
#metrics

y_pred_2_l = reg_svm.predict(X_test_2)
mse_2_l=mean_squared_error(y_test_2, y_pred_2_l, multioutput='uniform_average', squared=True)
print('MSE for predicting the DIRECTION OF MAXIMUM GAIN using linear SVR is ',end='')
print('%.5f'%(mse_2_l))


mae_2_l= mean_absolute_error(y_test_2, y_pred_2_l)
print('MAE for predicting the DIRECTION OF MAXIMUM GAIN using linear SVR is ',end='')
print('%.5f'%(mae_2_l))

MSE for predicting the DIRECTION OF MAXIMUM GAIN using linear SVR is 0.98735
MAE for predicting the DIRECTION OF MAXIMUM GAIN using linear SVR is 0.45855


### Support Vector Regression model with rbf kernel

### Default Parameters 


ML model : SVR with RBF kernel and default parameters  <br>
Prediction type: Regression <br>
Target Parameter: Maximum Gain <br>
Evaluation metrics: Mean Squared Error(MSE) , Mean absolute error(MAE)

In [21]:

reg_svm = SVR()
reg_svm.fit(X_train_1, y_train_1)


y_pred_1 = reg_svm.predict(X_test_1)
mse_1=mean_squared_error(y_test_1, y_pred_1 ,squared=True)

print('MSE for predicting the MAXIMUM GAIN using SVR with rbf kernel and default parameters is ',end="")
print('%.5f'%(mse_1))



mae_1= mean_absolute_error(y_test_1, y_pred_1)
print('MAE for predicting the MAXIMUM GAIN using SVR with rbf kernel and default parameters is  ', end='')
print('%.5f'%(mae_1))

MSE for predicting the MAXIMUM GAIN using SVR with rbf kernel and default parameters is 0.60732
MAE for predicting the MAXIMUM GAIN using SVR with rbf kernel and default parameters is  0.36300




<br>ML model : SVR with RBF kernel and default parameters  <br>
Prediction type: Regression <br>
Target Parameter: Direction of maximum Gain <br>
Evaluation metrics: Mean Squared Error(MSE) , Mean absolute error(MAE)

In [22]:
reg_svm = SVR()
reg_svm.fit(X_train_2, y_train_2)
#metrics

y_pred_2 = reg_svm.predict(X_test_2)
mse_2=mean_squared_error(y_test_2, y_pred_2, multioutput='uniform_average', squared=True)
print('MSE for predicting the DIRECTION OF MAXIMUM GAIN using SVR with rbf kernel and default parameters is ',end='') 
print('%.5f'%(mse_2))


mae_2= mean_absolute_error(y_test_2, y_pred_2)
print('MAE for predicting the DIRECTION OF MAXIMUM GAIN using SVR with rbf kernel and default parameters is ' ,end='')
print('%.5f'%(mae_2))


MSE for predicting the DIRECTION OF MAXIMUM GAIN using SVR with rbf kernel and default parameters is 0.50626
MAE for predicting the DIRECTION OF MAXIMUM GAIN using SVR with rbf kernel and default parameters is 0.37397



### Optimal Paramters 

In [23]:
param_grid=[
    {'C' : [1,10,100,1000],
     'gamma':['scale', 1, 0.1, 0.01],
     'kernel':['rbf'] },
]

In [24]:
optimal_params= GridSearchCV(
    SVR(),
    param_grid,
    cv=5,
    

)

<br>Determining the optimal parameters of the rbf kernel for determining the Maximum Gain

In [25]:
optimal_params.fit(X_train_1, y_train_1)
print(optimal_params.best_params_)
#Dictionary

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}


<br>Determining the optimal parameters of the rbf kernel for determining the Direction of Maximum Gain

In [26]:
optimal_params.fit(X_train_2, y_train_2)
print(optimal_params.best_params_)
#print('MAE: '+ str(optimal_params.best_score_))

{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}


<br>ML model : SVR with RBF kernel and optimal parameters  <br>
Prediction type: Regression <br>
Target Parameter: Maximum Gain <br>
Evaluation metrics: Mean Squared Error(MSE) , Mean absolute error(MAE)

In [27]:
reg_svm= SVR( C=10, gamma=0.01)
reg_svm.fit(X_train_1, y_train_1)
#metrics

y_pred_1_opt = reg_svm.predict(X_test_1)
mse_1_opt=mean_squared_error(y_test_1, y_pred_1_opt,squared=True)
print('MSE for predicting the MAXIMUM GAIN using SVR with rbf kernel and optimal parameters is ', end='')
print('%.5f'%(mse_1_opt))


mae_1_opt= mean_absolute_error(y_test_1, y_pred_1_opt)
print('MAE for predicting the MAXIMUM GAIN using SVR with rbf kernel and optimal parameters is ',end='')
print('%.5f'%(mae_1_opt))

MSE for predicting the MAXIMUM GAIN using SVR with rbf kernel and optimal parameters is 0.78608
MAE for predicting the MAXIMUM GAIN using SVR with rbf kernel and optimal parameters is 0.40410


In [28]:

# reg_svm= SVR( C=10, gamma=0.01)
# reg_svm.fit(X_train_1, y_train_1)
# y_pred_1_opt = reg_svm.predict(X_test_1)
# Comp_gain = pd.DataFrame({'Actual': (var_y_1*y_test_1) + mean_y_1, 'Predicted': (var_y_1*y_pred_1_opt) + mean_y_1})
# Comp_gain.head()


<br>ML model : SVR with RBF kernel and optimal parameters  <br>
Prediction type: Regression <br>
Target Parameter: Direction of Maximum Gain <br>
Evaluation metrics: Mean Squared Error(MSE) , Mean absolute error(MAE)

In [29]:
reg_svm= SVR( C=100, gamma=0.01)
reg_svm.fit(X_train_2, y_train_2)

#metrics

y_pred_2_opt = reg_svm.predict(X_test_2)
mse_2_opt=mean_squared_error(y_test_2, y_pred_2_opt,  squared=True)
print('MSE for predicting the DIRECTION_MAX_GAIN using SVR with rbf kernel and optimal parameters is ',end='')
print('%.5f'%(mse_2_opt))


mae_2_opt= mean_absolute_error(y_test_2, y_pred_2_opt)
print('MAE for predicting the DIRECTION_MAX_GAIN using SVR with rbf kernel and optimal parameters is ',end='')
print('%.5f'%(mae_2_opt))

MSE for predicting the DIRECTION_MAX_GAIN using SVR with rbf kernel and optimal parameters is 0.66713
MAE for predicting the DIRECTION_MAX_GAIN using SVR with rbf kernel and optimal parameters is 0.40831


In [30]:
# reg_svm= SVR( C=1000, gamma=0.01)
# reg_svm.fit(X_train_2, y_train_2)
# y_pred_2_opt = reg_svm.predict(X_test_2)
# Comp_gain = pd.DataFrame({'Actual': (var_y_2*y_test_2) + mean_y_2, 'Predicted': (var_y_2*y_pred_2_opt) + mean_y_2})
# Comp_gain
